In [1]:
# 목표: 주가 데이터 -> 캔들차트 이미지 변환 -> CNN 학습!
# 참고: https://github.com/quant4junior/algoTrade/blob/master/ch08/8.1%20CNN%EC%9D%84%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EC%BA%94%EB%93%A4%EC%B0%A8%ED%8A%B8%20%EC%98%88%EC%B8%A1%EB%B6%84%EC%84%9D/run_all_process.ipynb

In [2]:
######################################################
# 1.야후주식->데이터 다운로드->CSV저장
######################################################

# print(f'python run_binary_preprocessing.py "BBNI.JK" "20" "50"')
# subprocess.call(f'python run_binary_preprocessing.py  "BBNI.JK" "20" "50" ', shell=True)

!pip3 install arrow 

     |████████████████████████████████| 61kB 5.5MB/s 


In [3]:
# https://wendys.tistory.com/174
# 회사명으로 주식코드 받아오기

import pandas as pd
import pandas_datareader as pdr

# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    code = df.query("name=='{}'".format(name))['code'].to_string(index=False)
    # 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
    code = code.strip()
    return code

# excel 파일을 다운로드하는거와 동시에 pandas에 load하기
# 흔히 사용하는 df라는 변수는 data frame을 의미합니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
# data frame정리
code_df = code_df[['회사명', '종목코드']]
# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
# 종목코드는 6자리로 구분되기때문에 0을 채워 6자리로 변경
code_df.code = code_df.code.map('{:06d}'.format)
print(code_df.head())

# ex) 삼성전자의의 코드를 구해보겠습니다.
code = get_code(code_df, '삼성전자')
# yahoo의 주식 데이터 종목은 코스피는 .KS, 코스닥은 .KQ가 붙습니다.
# 삼성전자의 경우 코스피에 상장되어있기때문에 '종목코드.KS'로 처리하도록 한다.
code = code + '.KS'
print('code:', code)

# get_data_yahoo API를 통해서 yahho finance의 주식 종목 데이터를 가져온다.
df = pdr.get_data_yahoo(code)
print(df.head())

        name    code
0        DSR  155660
1      GS글로벌  001250
2  HDC현대산업개발  294870
3      KG케미칼  001390
4      LG이노텍  011070
code: 005930.KS
               High      Low     Open    Close      Volume     Adj Close
Date                                                                    
2016-03-09  23980.0  23540.0  23760.0  23880.0   9169000.0  20764.072266
2016-03-10  24720.0  24020.0  24160.0  24500.0  14177000.0  21303.173828
2016-03-11  25060.0  24320.0  24500.0  24980.0  13361250.0  21720.541016
2016-03-14  25460.0  24980.0  25340.0  25100.0  10857700.0  21824.882812
2016-03-15  25280.0  24920.0  25100.0  25060.0   8640400.0  21790.103516


In [4]:
import subprocess
import os
import sys

formatters = {
    'RED': '\033[91m',
    'GREEN': '\033[92m',
    'END': '\033[0m',
}

# symbol = sys.argv[1]
# windows_length = sys.argv[2]
# dimension = sys.argv[3]
# python run_binary_preprocessing.py "BBNI.JK" "20" "50"
# symbol = 'BBNI.JK' # JK: 인도네시아?

symbol = code # 삼성전자(005930.KS)
windows_length = '20'
dimension = '50'

training_start_date = "2020-01-01"
training_end_date = "2020-12-31"

testing_start_date = "2021-01-01"
testing_end_date = "2021-03-07"

download_data = True
onlytesting = True
onlytraining = True
use_volume = False

In [5]:
# subprocess.call(f'python utils/get_data.py -sd {testing_start_date} -ed {testing_end_date} -t {symbol} -s yahoo -p testing', shell=True)
import argparse
import arrow
import pandas as pd
import datetime as dt
from pandas_datareader import data, wb
import os
import fix_yahoo_finance as yf
import time
# fixed pandas_datareader can't download from yahoo finance
yf.pdr_override()


def get_data_main(prefix_name, start_date, end_date, ticker):
    print('--start get_data_main--')
    # parser = argparse.ArgumentParser(
    #     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('-sd', '--start_date', type=str,
    #                     default='1990-01-01', help='Start date parameter value - format YYYY-MM-DD')
    # parser.add_argument('-ed', '--end_date', type=str,
    #                     default=arrow.now().format('YYYY-MM-DD'), help='End date parameter - format YYYY-MM-DD')
    # parser.add_argument('-t', '--ticker', nargs='+',
    #                     help='<Required> Set flag', required=True)
    # parser.add_argument(
    #     '-s', '--source', help='<Required> set source', required=True)
    # parser.add_argument('-a', '--attempt',
    #                     help='set max attempt to download', default=10)
    # parser.add_argument(
    #     '-e', '--exist', help='check exist stock history file', default=False)
    # parser.add_argument('-p', '--prefix', help='add prefix in output name', default=is_test_tr)
    # args = parser.parse_args()
    # # fetch all data

    #prefix_name = ""
    # make sure output folder is exist
    if not os.path.isdir("/content/stockdatas"):
        os.mkdir("/content/stockdatas")
    # if len(args.prefix) > 1:
    #     prefix_name = args.prefix
    # if args.source == "tiingo":
    #     for ticker in set(args.ticker):
    #         fetch_tiingo_data(ticker, args.start_date, args.end_date,
    #                           "../stockdatas/{}_{}.csv".format(ticker, prefix_name))
    # elif args.source == "yahoo":
        #for ticker in set(args.ticker):
    #print('tickers:', tickers)
    #for ticker in set(tickers):
    print('ticker:', ticker)
    fetch_yahoo_data(ticker, start_date, end_date, "/content/stockdatas/{}_{}.csv".format(ticker, prefix_name), 3, False)
    print('--end get_data_main--')


def fetch_tiingo_data(ticker, start_date, end_date, fname):
    url = "https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={start_date}&endDate={end_date}&token={token}"
    token = "ca5a6f47a99ae61051e4de63b26f727b1709a01d"
    data = pd.read_json(url.format(
        ticker=ticker,
        start_date=start_date,
        end_date=end_date,
        token=token
    ))
    data.to_csv(fname, columns=["date", "open", "close",
                                "high", "low", "volume", "adjClose"], index=False)


def fetch_yahoo_data(ticker, start_date, end_date, fname, max_attempt, check_exist):
    if (os.path.exists(fname) == True) and check_exist:
        print("file exist")
    else:
        # remove exist file
        if os.path.exists(fname):
            os.remove(fname)
        for attempt in range(max_attempt):
            time.sleep(2)
            try:
                dat = data.get_data_yahoo(''.join("{}".format(
                    ticker)),  start=start_date, end=end_date)
                dat.to_csv(fname)
            except Exception as e:
                if attempt < max_attempt - 1:
                    print('Attempt {}: {}'.format(attempt + 1, str(e)))
                else:
                    raise
            else:
                break

In [6]:
# 주식 데이터 다운로드
try:
    if download_data:
        if onlytesting:
            # get data testing
            print('{RED}\nGet Testing Data{END}'.format(**formatters))
            #subprocess.call(f'python utils/get_data.py -sd {testing_start_date} -ed {testing_end_date} -t {symbol} -s yahoo -p testing', shell=True)
            get_data_main('testing', testing_start_date, testing_end_date, symbol)
            
            print('{GREEN}Get Testing Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print(identifier)

try:
    if download_data:
        if onlytraining:
            # get data testing
            print('{RED}\nGet Training Data{END}'.format(**formatters))
            #subprocess.call(f'python utils/get_data.py -sd {training_start_date} -ed {training_end_date} -t {symbol} -s yahoo -p training', shell=True)
            get_data_main('training', training_start_date, training_end_date, symbol)

            print('{GREEN}Get Training Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print(identifier)


Get Testing Data
--start get_data_main--
ticker: 005930.KS
[*********************100%***********************]  1 of 1 downloaded
--end get_data_main--
Get Testing Data Done


Get Training Data
--start get_data_main--
ticker: 005930.KS
[*********************100%***********************]  1 of 1 downloaded
--end get_data_main--
Get Training Data Done



In [ ]:
######################################################
# 2.CSV->데이터별 라벨링
# 3.2.CSV->업다운(1,0)->특정주기로 이미지(캔들)로저장
# 3.이미지/라벨별 -> 폴더이동
######################################################

In [7]:
!pip3 install mpl-finance

In [8]:
# python preproccess_binclass.py -m createLabel -l {windows_length} -i stockdatas/{symbol}_training.csv
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import glob
import argparse
import os
from shutil import copyfile, move
from pathlib import Path

# https://github.com/matplotlib/mpl_finance
from mpl_finance import candlestick2_ochl, volume_overlay


def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False


def removeOutput(finput):
    if(Path(finput)).is_file():
        os.remove(finput)


def preproccess_binclass_main(mode, seq_len, input, dataset_type, dimension, use_volume, label_file=''):
    # parser = argparse.ArgumentParser(
    #     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('-i', '--input',
    #                     help='a csv file of stock data', required=True)
    # parser.add_argument('-l', '--seq_len',
    #                     help='num of sequence length', default=20)
    # parser.add_argument('-lf', '--label_file',
    #                     help='a label_file')
    # parser.add_argument('-d', '--dimension',
    #                     help='a dimension value', type=int, default=48)
    # parser.add_argument('-t', '--dataset_type',
    #                     help='training or testing datasets')
    # parser.add_argument('-m', '--mode',
    #                     help='mode of preprocessing data', required=True)
    # parser.add_argument('-v', '--use_volume',
    #                     help='combine with volume.', default=False)
    # args = parser.parse_args()

    print('seq_len:', seq_len, ' ,input:', input)
    if mode == 'ohlc2cs':
        ohlc2cs(input, seq_len, dataset_type, dimension, use_volume)
    if mode == 'createLabel':
        createLabel(input, seq_len)
    if mode == 'img2dt':
        image2dataset(input, label_file)
    if mode == 'countImg':
        countImage(input)

def createLabel(fname, seq_len):
    # fname: /content/stockdatas/005930.KS_training.csv

    # python preprocess.py -m createLabel -l 20 -i stockdatas/EWT_training5.csv
    print("Creating label . . .")
    # remove existing label file
    filename = fname.split('/')
    
    #print("{} - {}".format(filename[0], filename[1][:-4]))
    print("{} - {}".format(filename[2], filename[3][:-4]))
    removeOutput("{}_label_{}.txt".format(filename[3][:-4], seq_len))

    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)

    df.reset_index(inplace=True)
    df['Date'] = df['Date'].map(mdates.date2num)

    print('len(df):', len(df))
    for i in range(0, len(df)):
        #c = df.ix[i:i + int(seq_len), :]
        c = df.iloc[i:i + int(seq_len), :]

        starting = 0
        endvalue = 0
        label = ""
        
        #print('len(c):', len(c), ' ,int(seq_len)+1:', int(seq_len)+1)
        #if len(c) == int(seq_len)+1:  ## 여기서 안나왔겠지. 마지막 부분이. 3450 3449
        if len(c) == int(seq_len): # +1할 이유가 없음, ix대신 iloc를 사용해서 그런듯?
            # starting = c["Close"].iloc[-2]
            starting = c["Open"].iloc[-1]
            endvalue = c["Close"].iloc[-1]
            # print(f'endvalue {endvalue} - starting {starting}')
            tmp_rtn = endvalue / starting -1
            if tmp_rtn > 0:
                label = 1
            else:
                label = 0

            with open("{}_label_{}.txt".format(filename[3][:-4], seq_len), 'a') as the_file:
                the_file.write("{}-{},{}".format(filename[3][:-4], i, label))
                the_file.write("\n")
            #print('file create success!')
    print("Create label finished.")


def countImage(input):
    num_file = sum([len(files) for r, d, files in os.walk(input)])
    num_dir = sum([len(d) for r, d, files in os.walk(input)])
    print("num of files : {}\nnum of dir : {}".format(num_file, num_dir))


def ohlc2cs(fname, seq_len, dataset_type, dimension, use_volume):
    # python preprocess.py -m ohlc2cs -l 20 -i stockdatas/EWT_testing.csv -t testing
    print("Converting ohlc to candlestick")
    symbol = fname.split('_')[0]
    symbol = symbol.split('/')[3]
    print('symbol:', symbol)
    path = "{}".format(os.getcwd())
    # print(path)
    if not os.path.exists("{}/dataset/{}_{}/{}/{}".format(path, seq_len, dimension, symbol, dataset_type)):
        os.makedirs("{}/dataset/{}_{}/{}/{}".format(path,seq_len, dimension, symbol, dataset_type))

    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)
    plt.style.use('dark_background')
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].map(mdates.date2num)
    # for i in range(0, len(df)):

    dimension = int(dimension)
    for i in range(0, len(df)-int(seq_len)):
        # ohlc+volume
        #c = df.ix[i:i + int(seq_len) - 1, :]
        c = df.iloc[i:i + int(seq_len), :] # ix->iloc로 바뀌고 -1 불필요??

        #print('len(c):', len(c), ' ,int(seq_len):', int(seq_len))
        if len(c) == int(seq_len):
            my_dpi = 96
            fig = plt.figure(figsize=(dimension / my_dpi,
                                      dimension / my_dpi), dpi=my_dpi)
            ax1 = fig.add_subplot(1, 1, 1)
            candlestick2_ochl(ax1, c['Open'], c['Close'], c['High'],c['Low'],
                              width=1,colorup='#77d879', colordown='#db3f3f')
            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')

            # create the second axis for the volume bar-plot
            # Add a seconds axis for the volume overlay
            if use_volume:
                ax2 = ax1.twinx()
                # Plot the volume overlay
                bc = volume_overlay(ax2, c['Open'], c['Close'], c['Volume'],
                                    colorup='#77d879', colordown='#db3f3f', alpha=0.5, width=1)
                ax2.add_collection(bc)
                ax2.grid(False)
                ax2.set_xticklabels([])
                ax2.set_yticklabels([])
                ax2.xaxis.set_visible(False)
                ax2.yaxis.set_visible(False)
                ax2.axis('off')
            pngfile = 'dataset/{}_{}/{}/{}/{}-{}.png'.format(
                seq_len, dimension, symbol, dataset_type, symbol+"_"+dataset_type, i)
            fig.savefig(pngfile, pad_inches=0, transparent=False)
            plt.close(fig)

            # Alpha 채널 없애기 위한.
            from PIL import Image
            img = Image.open(pngfile)
            img = img.convert('RGB')
            img.save(pngfile)

        # normal length - end

    print("Converting olhc to candlestik finished.")


def image2dataset(input, label_file):    
    label_dict = {}
    with open(label_file) as f:
        for line in f:
            (key, val) = line.split(',')
            label_dict[key] = val.rstrip()
    
    path = "{}/{}".format(os.getcwd(), input)
    print('path:', path)

    for filename in os.listdir(path):
        # print(filename)
        # print(os.getcwd())
        if filename is not '':
            for k, v in label_dict.items():
                splitname = filename.split("_")
                f, e = os.path.splitext(filename)
                # print("[DEBUG] - {}".format(splitname))
                newname = "{}_{}".format(splitname[0], splitname[1])
                if newname == k:
                    print("{} same with {} with v {}".format(filename, k, v))
                    new_name = "{}{}.png".format(v, f)

                    os.rename("{}/{}".format(path, filename),
                              "{}/{}".format(path, new_name))
                    break

    folders = ['1', '0']
    for folder in folders:
        if not os.path.exists("{}/classes/{}".format(path, folder)):
            os.makedirs("{}/classes/{}".format(path, folder))
            print('creat folder:', "{}/classes/{}".format(path, folder))
    
    for filename in os.listdir(path):
        try:
            if filename is not '' and filename is not 'classes' and filename!='classes':
                ### 여기에 for k,v in label_dict.items() 돌면서
                f, e = os.path.splitext(filename)
                #print('filename:',filename, ',f:',f, 'e:',e)

                if label_dict[f] == "1":
                    move("{}/{}".format(path, filename),
                        "{}/classes/1/{}".format(path, filename))
                elif label_dict[f] == "0":
                    move("{}/{}".format(path, filename),
                        "{}/classes/0/{}".format(path, filename))
        except Exception as e:
            print('file move err:', e)

    print('Done')

In [9]:
# 2.CSV->데이터별 라벨링
try:
    if onlytraining:
        # create label training
        print('{RED}\nCreate Label Training Data{END}'.format(**formatters))
        #subprocess.call(f'python preproccess_binclass.py -m createLabel -l {windows_length} -i stockdatas/{symbol}_training.csv', shell=True)
        preproccess_binclass_main('createLabel', windows_length, '/content/stockdatas/'+symbol+'_training.csv', '','','')

        print('{GREEN}Create Label Training Data Done\n{END}!'.format(**formatters))
    if onlytesting:
        # create label testing
        print('{RED}\nCreate Label Testing Data{END}'.format(**formatters))
        #subprocess.call(f'python preproccess_binclass.py -m createLabel -l {windows_length} -i stockdatas/{symbol}_testing.csv', shell=True)
        preproccess_binclass_main('createLabel', windows_length, '/content/stockdatas/'+symbol+'_testing.csv', '','','')

        print('{GREEN}Create Label Testing Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print(identifier)


Create Label Training Data
seq_len: 20  ,input: /content/stockdatas/005930.KS_training.csv
Creating label . . .
stockdatas - 005930.KS_training
len(df): 248
Create label finished.
Create Label Training Data Done
!

Create Label Testing Data
seq_len: 20  ,input: /content/stockdatas/005930.KS_testing.csv
Creating label . . .
stockdatas - 005930.KS_testing
len(df): 42
Create label finished.
Create Label Testing Data Done



In [10]:
# 3.2.CSV->업다운(1,0)->특정주기로 이미지(캔들)로저장
ㅌtry:
    if onlytraining:
        # convert to candlestick chart training data
        print('{RED}\nConvert Training Data to Candlestik{END}'.format(**formatters))
        #subprocess.call(f'python preproccess_binclass.py -m ohlc2cs -l {windows_length} -i stockdatas/{symbol}_training.csv -t training -d {dimension} -v {use_volume}', shell=True)
        preproccess_binclass_main('ohlc2cs', windows_length, '/content/stockdatas/'+symbol+'_training.csv', 'training',dimension,'False')

        print('{GREEN}Convert Training Data to Candlestik Done\n{END}'.format(
            **formatters))
    if onlytesting:
        # convert to candlestick chart testing data
        print('{RED}\nConvert Testing Data to Candlestik{END}'.format(**formatters))
        #subprocess.call(f'python preproccess_binclass.py -m ohlc2cs -l {windows_length} -i stockdatas/{symbol}_testing.csv -t testing -d {dimension} -v {use_volume}', shell=True)
        preproccess_binclass_main('ohlc2cs', windows_length, '/content/stockdatas/'+symbol+'_testing.csv', 'testing',dimension,'False')
        
        print('{GREEN}Convert Testing Data to Candlestik Done\n{END}'.format(
            **formatters))
except Exception as identifier:
    print(identifier)


Convert Training Data to Candlestik
seq_len: 20  ,input: /content/stockdatas/005930.KS_training.csv
Converting ohlc to candlestick
symbol: 005930.KS
Converting olhc to candlestik finished.
Convert Training Data to Candlestik Done


Convert Testing Data to Candlestik
seq_len: 20  ,input: /content/stockdatas/005930.KS_testing.csv
Converting ohlc to candlestick
symbol: 005930.KS
Converting olhc to candlestik finished.
Convert Testing Data to Candlestik Done



In [11]:
# 3.이미지/라벨별 -> 폴더이동
try:
    if onlytraining:
        # labelling data training
        print('{RED}\nLabelling Training Data{END}'.format(**formatters))
        print(f'python preproccess_binclass.py -m img2dt -i dataset/{windows_length}_{dimension}/{symbol}/training -lf {symbol}_training_label_{windows_length}.txt')
        #subprocess.call(f'python preproccess_binclass.py -m img2dt -i dataset/{windows_length}_{dimension}/{symbol}/training -lf {symbol}_training_label_{windows_length}.txt', shell=True)
        preproccess_binclass_main('img2dt', 0, 'dataset/20_50/'+symbol+'/training', '','','', '/content/'+symbol+'_training_label_20.txt')
        
        print('{GREEN}Labelling Training Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print('err:', identifier)

try:
    if onlytesting:
        # labelling data testing
        print('{RED}\nLabelling Testing Data{END}'.format(**formatters))
        #subprocess.call(f'python preproccess_binclass.py -m img2dt -i dataset/{windows_length}_{dimension}/{symbol}/testing -lf {symbol}_testing_label_{windows_length}.txt', shell=True)
        preproccess_binclass_main('img2dt', 0, 'dataset/20_50/'+symbol+'/testing', '','','', '/content/'+symbol+'_testing_label_20.txt')
        
        print('{GREEN}Labelling Testing Data Done\n{END}'.format(**formatters))
except Exception as identifier:
    print('err:', identifier)


Labelling Training Data
python preproccess_binclass.py -m img2dt -i dataset/20_50/005930.KS/training -lf 005930.KS_training_label_20.txt
seq_len: 0  ,input: dataset/20_50/005930.KS/training
path: /content/dataset/20_50/005930.KS/training
creat folder: /content/dataset/20_50/005930.KS/training/classes/1
creat folder: /content/dataset/20_50/005930.KS/training/classes/0
Done
Labelling Training Data Done


Labelling Testing Data
seq_len: 0  ,input: dataset/20_50/005930.KS/testing
path: /content/dataset/20_50/005930.KS/testing
creat folder: /content/dataset/20_50/005930.KS/testing/classes/1
creat folder: /content/dataset/20_50/005930.KS/testing/classes/0
Done
Labelling Testing Data Done



In [12]:
# !\rm -rf /content/dataset/20_50/005930.KS/testing/classes/
# !\rm -rf /content/dataset/20_50/005930.KS/training/classes/
# #!ls -alrt /content/dataset/20_50/005930.KS/training/

In [ ]:
# 4.학습전 data 복제
# print(f'python generatedata.py "dataset" "20_50/BBNI.JK" "dataset_BBNIJK_20_50" ')
# subprocess.call(f'python generatedata.py "dataset" "20_50/BBNI.JK" "dataset_BBNIJK_20_50" ', shell=True)

In [27]:
import os
from shutil import copyfile
import sys

def cre8outputdir(pathdir, targetdir):
    # create folder output
    if not os.path.exists("{}/{}".format(pathdir, targetdir)):
        os.mkdir("{}/{}".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/train".format(pathdir, targetdir)):
        os.mkdir("{}/{}/train".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/test".format(pathdir, targetdir)):
        os.mkdir("{}/{}/test".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/train/0".format(pathdir, targetdir)):
        os.mkdir("{}/{}/train/0".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/train/1".format(pathdir, targetdir)):
        os.mkdir("{}/{}/train/1".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/test/0".format(pathdir, targetdir)):
        os.mkdir("{}/{}/test/0".format(pathdir, targetdir))

    if not os.path.exists("{}/{}/test/1".format(pathdir, targetdir)):
        os.mkdir("{}/{}/test/1".format(pathdir, targetdir))

# pathdir = sys.argv[1]
# origindir = sys.argv[2]
# targetdir = sys.argv[3]
pathdir = 'dataset'
origindir = '20_50/' + symbol # 005930.KS
targetdir = 'dataset_' + symbol + '20_50'

cre8outputdir(pathdir, targetdir)

counttest = 0
counttrain = 0
for root, dirs, files in os.walk("{}/{}".format(pathdir, origindir)):
    for file in files:


        tmp = root.replace('\\','/')
        tmp_label = tmp.split('/')[-1]

        if tmp_label == '0':
            if 'test' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/test/0/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttest += 1
            elif 'train' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/train/0/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttrain += 1
        elif tmp_label == '1':
            if 'test' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/test/1/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttest += 1
            elif 'train' in file:
                origin = "{}/{}".format(root, file)
                destination = "{}/{}/train/1/{}".format(
                    pathdir, targetdir, file)
                copyfile(origin, destination)
                counttrain += 1

print(counttest)
print(counttrain)

22
228


In [13]:
!apt-get install zip unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version (3.0-11build1).
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [29]:
#압축 100M이하로! : !zip -s 100M
!zip -r dataset_zip.zip /content/dataset

  adding: content/dataset/ (stored 0%)
  adding: content/dataset/20_50/ (stored 0%)
  adding: content/dataset/20_50/005930.KS/ (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/ (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/ (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/ (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testing-16.png (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testing-5.png (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testing-1.png (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testing-9.png (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testing-10.png (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testing-13.png (stored 0%)
  adding: content/dataset/20_50/005930.KS/testing/classes/1/005930.KS_testi